# Tensor Fusion Model - CMU-MOSEI Dataset

In this notebook, we implement the Tensor Fusion Model, and test it on CMU-MOSEI dataset.

## Importing Libraries

In [11]:
import os
import sys
import torch
from torch import nn

import numpy as np
import matplotlib.pyplot as plt

sys.path.append(os.getcwd())
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

## Importing Helper Functions

In [12]:
from helpers.data.get_data import get_dataloader
from helpers.unimodals.common_models import GRUWithLinear, MLP
from helpers.fusions.common_fusions import Concat, TensorFusion
from helpers.training.all_in_one import all_in_one_train
from helpers.training.supervised_learning import train, test
from helpers.utils.common_utils import get_default_device, save_results

## Loading the Dataset

In [13]:
traindata, validdata, testdata = get_dataloader(
    'data/mosei_senti_data.pkl', robust_test=False)

In [14]:
device = get_default_device()
print(device)

cuda


## Model Architecture

In [15]:
encoders = [GRUWithLinear(35, 64, 4, dropout=True, has_padding=True).to(device),
            GRUWithLinear(74, 128, 19, dropout=True, has_padding=True).to(device),
            GRUWithLinear(300, 512, 79, dropout=True, has_padding=True).to(device)]

head = MLP(8000, 512, 1).to(device)

fusion = TensorFusion().to(device)

## Training

In [16]:
FILE_NAME = 'mosei_tfus_r0'

In [ ]:
train_losses, valid_losses = train(
    encoders, fusion, head, traindata, validdata, 100, 
    task="regression", optimtype=torch.optim.AdamW,
    early_stop=False, is_packed=True, lr=1e-3, 
    save=f'models/{FILE_NAME}.pt', 
    weight_decay=0.01, objective=torch.nn.L1Loss()
)

Epoch [1/100]
Train Loss: 0.7575
Valid Loss: 0.6296
Saving Best

Epoch [2/100]
Train Loss: 0.6464
Valid Loss: 0.6113
Saving Best

Epoch [3/100]
Train Loss: 0.6104
Valid Loss: 0.6603

Epoch [4/100]
Train Loss: 0.5696
Valid Loss: 0.6145

Epoch [5/100]
Train Loss: 0.5370
Valid Loss: 0.5949
Saving Best

Epoch [6/100]
Train Loss: 0.4965
Valid Loss: 0.5907
Saving Best

Epoch [7/100]
Train Loss: 0.4534
Valid Loss: 0.6244

Epoch [8/100]
Train Loss: 0.4084
Valid Loss: 0.6032

Epoch [9/100]
Train Loss: 0.3721
Valid Loss: 0.6054

Epoch [10/100]
Train Loss: 0.3350
Valid Loss: 0.6048

Epoch [11/100]
Train Loss: 0.3132
Valid Loss: 0.6057

Epoch [12/100]
Train Loss: 0.2907
Valid Loss: 0.6063

Epoch [13/100]
Train Loss: 0.2692
Valid Loss: 0.6017

Epoch [14/100]
Train Loss: 0.2557
Valid Loss: 0.6062

Epoch [15/100]
Train Loss: 0.2470
Valid Loss: 0.6270

Epoch [16/100]
Train Loss: 0.2344
Valid Loss: 0.6026

Epoch [17/100]
Train Loss: 0.2276
Valid Loss: 0.6043

Epoch [18/100]
Train Loss: 0.2156
Valid Los

In [ ]:
plt.plot(train_losses, label='Train Loss')
plt.plot(valid_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.title('Loss vs. No. of epochs')

In [ ]:
save_results(train_losses, valid_losses, f'results/{FILE_NAME}.pkl')

## Testing

In [ ]:
model = torch.load(f'models/{FILE_NAME}.pt').to(device)

test(model, test_dataloaders_all=testdata, dataset='mosi',
     is_packed=True, criterion=torch.nn.L1Loss(), task='posneg-classification', no_robust=True)